In [1]:
import dive as dive
import pymc3 as pm
import deerlab as dl
import numpy as np
%load_ext autoreload
%autoreload 2

/opt/anaconda3/envs/pymc_old/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def preprocess(file_name):
    (t, idx), V = dl.deerload(file_name)
    V = dl.correctphase(V)
    V = V[:,V.sum(axis=0) != 0][:,:-1].sum(axis=1)
    t = dl.correctzerotime(V, t)
    
    return V, t

In [3]:
t = np.linspace(-0.1,5,300)

r = np.linspace(2,15,400)           # distance axis, nm
param = [5,0.2,0.4,5.4,0.15,0.6] # parameters for single-Gaussian model
P = dl.dd_gauss2(r,param)          # model distance distribution
lam = 0.4                       # modulation depth
B = dl.bg_hom3d(t,0.2,lam)         # background decay
K = dl.dipolarkernel(t,r,mod=lam,bg=B)  # kernel matrix

Vexp = K@P + dl.whitegaussnoise(t,0.01,seed=0)
def generatedata(Vexp):
    return t, Vexp, r, P,K


t, Vexp, r, P,K= generatedata(Vexp)

t1 = t
V1 = Vexp
r1 = r
P1 = P




In [5]:
pars = {"method": 'gaussian', "r": np.linspace(2,7,100), "nGauss": 1}

MCMCparameters = {"draws": 5000,"tune": 2000, "chains": 4, "cores": 4}
NUTSpars = {"target_accept":0.95}

Model = dive.model(t1, V1, pars)

#trace,model =  dive.sample(Model, MCMCparameters,  NUTSpars=NUTSpars,seed=True, seeds=[1,2,3,4])
trace,model =  pm.sample(Model, MCMCparameters,  NUTSpars=NUTSpars)

Time-domain data:    300 points from -0.1 µs to 5 µs
Distance vector:     100 points from 2 nm to 7 nm
Model:              gaussian
Number of Gaussian: 1


TypeError: No model on context stack.

In [10]:
Model = dive.model(t1, V1, pars)

Model = Model.get_context()




Time-domain data:    300 points from -0.1 µs to 5 µs
Distance vector:     100 points from 2 nm to 7 nm
Model:              gaussian
Number of Gaussian: 1


AttributeError: 'dict' object has no attribute 'get_context'

In [11]:
type(Model)

dict

In [17]:
with Model['model']

In [35]:
keys

[]

In [9]:
tis

{}